# Predicting Automobile Prices

# Import and View the Data

In [ ]:
# Import pandas library

import pandas as pd

# Read automobile data csv file

path = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DA0101EN/auto.csv"
df = pd.read_csv(path, header=None)
df.head(10)

In [ ]:
# Add the headers since they are stored in a separate file

# create headers list

headers = ["symboling","normalized-losses","make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","wheel-base", "length","width","height","curb-weight","engine-type",
         "num-of-cylinders", "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower",
         "peak-rpm","city-mpg","highway-mpg","price"]
print("headers\n", headers)

In [ ]:
# Add the headers and recheck the dataframe

df.columns = headers
df.head(10)

In [ ]:
# View the data types of variables in the dataframe

df.dtypes

In [ ]:
# View summary statistics for each variable, include sring/object variables

df.describe(include='all')

# Handle Missing Data

In [ ]:
import numpy as np

# replace "?" with NaN in variables

df.replace("?", np.nan, inplace = True)
df.head(5)

In [ ]:
# Show number of missing values in each variable

df.isnull().sum()

In [ ]:
# Fill missing values with the average of the given variable:

fillna = ["normalized-losses", "bore", "stroke", "horsepower", "peak-rpm", "price"]

for series in fillna:
    series_avg = df[series].astype('float').mean(axis=0)
    df[series].replace(np.nan, series_avg, inplace=True)

In [ ]:
# For variable num-of-doors, replace missings with the most frequent value 'four' shown in the descriptives

df["num-of-doors"].replace(np.nan, "four", inplace=True)

In [ ]:
# Verify that no missings are left

df.isna().sum()

# Correct Data Formats

In [ ]:
# Revisit data types and descriptives to identify strings/objects for conversion to numeric

print(df.dtypes)

df.describe(include='all')

In [ ]:
# Convert numeric variables saved as string/object to numeric 

df[["normalized-losses"]] = df[["normalized-losses"]].astype("int")
df[["bore", "stroke", "horsepower", "peak-rpm", "price"]] = df[["bore", "stroke", "horsepower", "peak-rpm", "price"]].astype("float")

In [ ]:
# Verify conversion

print(df.dtypes)
df.describe(include="all")

# Bin Continuous Variables

In [ ]:
# Binning select variables

# Convert to needed format: from float to integer

df["horsepower"]=df["horsepower"].astype(int, copy=True)

In [ ]:
# View histogram of horsepower to determine bin sizes

%matplotlib inline
import matplotlib as plt
from matplotlib import pyplot
plt.pyplot.hist(df["horsepower"])

# set x/y labels and title

plt.pyplot.xlabel("horsepower")
plt.pyplot.ylabel("count")
plt.pyplot.title("horsepower bins")


In [ ]:
# Create 3 bins of equal size, i.e, there are 4 dividers that are cut-off values for the bins. 
# Store the cut-off points in array "bins"

bins = np.linspace(min(df["horsepower"]), max(df["horsepower"]), 4)
print(bins)

# Name the bins

group_names = ['Low', 'Medium', 'High']

In [ ]:
# Assign bins to horsepower values with the 'cut' function

df['horsepower-binned'] = pd.cut(df['horsepower'], bins, labels=group_names, include_lowest=True )
df[['horsepower','horsepower-binned']].head(20)

In [ ]:
# View number of autos in each bin

df["horsepower-binned"].value_counts()

In [ ]:
# Visualize the bins in a histogram

plt.pyplot.hist(df["horsepower"], bins = 3)

# set x/y labels and title
plt.pyplot.xlabel("horsepower")
plt.pyplot.ylabel("count")
plt.pyplot.title("horsepower bins")

# Normalize Continuous Variables

In [ ]:
# Extract the continuous variables 

df_cont = df.select_dtypes(include=['float64', 'int64'])

df_cont

In [ ]:
# Plot the first non-normalized variable 

plt.pyplot.plot(df['normalized-losses'])
plt.pyplot.ylabel('normalized losses')
plt.pyplot.show()

In [ ]:
# List the continuous variables/columns

df_cont.columns

In [ ]:
# Normalize all continous vars in the list

to_norm = ['symboling', 'normalized-losses', 'wheel-base', 'length', 'width',
       'height', 'curb-weight', 'engine-size', 'bore', 'stroke',
       'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
       'highway-mpg', 'price']

for series in to_norm:
    df[series] = df[series] / df[series].max() 
    

In [ ]:
# Verify variables are on normalized scale

df.describe()

In [ ]:
# Plot of first normalized variable shows the same dynamics as above, now on the rescaled Y-axis

plt.pyplot.plot(df['normalized-losses'])
plt.pyplot.ylabel('normalized losses')
plt.pyplot.show()


# Dichotomize Categorical Variables into Dummies

In [ ]:
# Extract the categorical variables

df_cat = df.select_dtypes(include=['object'])
df_cat.columns

In [ ]:
# Create dummies from categorical variables of interest

df_d1= pd.get_dummies(df['make'])
df_d2= pd.get_dummies(df['fuel-type'])
df_d3= pd.get_dummies(df['num-of-doors'])
df_d4= pd.get_dummies(df['body-style'])
df_d5= pd.get_dummies(df['engine-type'])

In [ ]:
# merge the main dataframe with the new dummy variable dataframes

df = pd.concat([df, df_d1, df_d2, df_d3, df_d4, df_d5], axis=1)

# Explore Data to Select Model

In [ ]:
# Compute pair-wise correlations of each feature with target variable price

corr_list = df[df.columns[1:]].corr()['price'][:]
corr_table = pd.DataFrame(corr_list)

# Sort the correlations

corr_table_s = corr_table.sort_values(by='price', ascending=False)

# View top 5 positively correlated features

corr_table_s.head(6)

In [ ]:
# View top 5 negatively correlated features

corr_table_s.tail(5)

In [ ]:
# Visually examine top correlated features and price in scatter plots  

%matplotlib inline
import matplotlib as plt
from matplotlib import pyplot

to_scatter = ['engine-size', 'curb-weight', 'horsepower', 'width', 'length', 'highway-mpg', 'city-mpg']

for series in to_scatter:
    x = df[series]
    y = df["price"]
    plt.pyplot.scatter(x, y)
    plt.pyplot.xlabel(series)
    plt.pyplot.ylabel("price")
    plt.pyplot.show()


In [ ]:
# Take logs of top features and price; 

df['engine-size_l'] = np.log(df['engine-size'])
df['curb-weight_l'] = np.log(df['curb-weight'])
df['horsepower_l'] = np.log(df['horsepower'])
df['width_l'] = np.log(df['width'])
df['length_l'] = np.log(df['length'])
df['highway-mpg_l'] = np.log(df['highway-mpg'])
df['city-mpg_l'] = np.log(df['city-mpg'])
df['price_l'] = np.log(df['price'])

# Plot the log features with log of price; plots show linearity in logs

to_scatter = ['engine-size_l', 'curb-weight_l', 'horsepower_l', 'width_l', 'length_l', 'highway-mpg_l', 'city-mpg_l']

for series in to_scatter:
    x = df[series]
    y = df["price_l"]
    plt.pyplot.scatter(x, y)
    plt.pyplot.xlabel(series)
    plt.pyplot.ylabel("price_l")
    plt.pyplot.show()

In [ ]:
# Re-calculate linear correlations with p-values for statistical significance for top features in logs

from scipy import stats


selected_features = ['engine-size_l', 'curb-weight_l', 'horsepower_l', 'width_l', 'length_l', 'highway-mpg_l', 'city-mpg_l']

for series in selected_features:

    pearson_coef, p_value = stats.pearsonr(df[series], df['price_l'])
    print("The Pearson Correlation of log of price with ", series, "is ", round(pearson_coef,4), " with P-value =", round(p_value,4))



In [ ]:
# Calculate Correlation Matrix for top features to check for multicollinearity.

corr_list = df[['engine-size_l', 'curb-weight_l', 'horsepower_l', 'highway-mpg_l', 'city-mpg_l']]

corr = corr_list.corr()
corr

# Explore Bivariate Regressions

In [ ]:
# Import the visualization package: seaborn

import seaborn as sns


In [ ]:
# Bivariate Regression Plots (i.e, scatter plots with fitted lines)

to_regplots = ['engine-size_l', 'curb-weight_l', 'horsepower_l', 'highway-mpg_l']

for series in to_regplots:
    width = 6
    height = 4
    plt.pyplot.figure(figsize=(width, height))
    sns.regplot(x=df[series], y=df['price_l'], data=df)

In [ ]:
# Bivariate Residual Plots (should be evenly distributed around zero)

to_resplots = ['engine-size_l', 'curb-weight_l', 'horsepower_l', 'highway-mpg_l']

for series in to_resplots:
    width = 6
    height = 4
    plt.pyplot.figure(figsize=(width, height))
    sns.residplot(df[series], df['price_l'])
    plt.pyplot.show()


# Train Multiple Regression Model

In [ ]:
# Train model on final selected features with strongest linear relationship with price

from sklearn.linear_model import LinearRegression

lm = LinearRegression()

X = df[['engine-size_l', 'horsepower_l', 'highway-mpg_l']]

lm.fit(X, df['price_l'])


print(lm.intercept_)
print(lm.coef_)



# Predict Auto Prices

In [ ]:
# Predict, i.e, Calculate auto prices using the regression equation

Y_hat = lm.predict(X)

In [ ]:
# Distribution Plot of Actual vs Predicted Prices

plt.pyplot.figure(figsize=(width, height))


ax1 = sns.distplot(df['price_l'], hist=False, color="r", label="Actual Value")
sns.distplot(Y_hat, hist=False, color="b", label="Fitted Values" , ax=ax1)


plt.pyplot.title('Actual vs Fitted Values for Log Price')
plt.pyplot.xlabel('Price (in logs)')
plt.pyplot.ylabel('Proportion of Cars')

plt.pyplot.show()
plt.pyplot.close()

# Evaluate Model Performance

In [ ]:
# Find the R^2

print('The R-square is: ', lm.score(X, df['price_l']))

In [ ]:
# Find MSE (Mean Squared Error), i.e, error between calculated/predicted and actual target Price

from sklearn.metrics import mean_squared_error

print('The mean square error of log price and predicted log price using multifit is: ', \
      mean_squared_error(df['price_l'], Y_hat))

# Validate Model

In [ ]:
# Place target data in separate dataframe

y_data = df['price_l']

In [ ]:
# Drop Y target data from X data

x_data=df.drop('price_l', axis=1)
x_data=df.drop('price', axis=1)

In [ ]:
# Split data into train and test data

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.30, random_state=1)


print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])

In [ ]:
# Train the model on the train data

from sklearn.linear_model import LinearRegression

lre=LinearRegression()

lre.fit(x_train[['engine-size_l', 'horsepower_l', 'highway-mpg_l']], y_train)

In [ ]:
# Get R^2 of model on train data

lre.score(x_train[['engine-size_l', 'horsepower_l', 'highway-mpg_l']], y_train)

In [ ]:
# Test the model on the test data

lre.fit(x_test[['engine-size_l', 'horsepower_l', 'highway-mpg_l']], y_test)

In [ ]:
# Get R^2 of model on test data

lre.score(x_test[['engine-size_l', 'horsepower_l', 'highway-mpg_l']], y_test)

# Cross-Validation

In [ ]:
# Split data in 4 folds (cv=4) and calculate R^2 for each fold

from sklearn.model_selection import cross_val_score

Rcross = cross_val_score(lre, x_data[['engine-size_l', 'horsepower_l', 'highway-mpg_l']], y_data, cv=4)

Rcross


In [ ]:
print("The mean of the folds is", Rcross.mean(), "and the standard deviation is" , Rcross.std())

# Visualize Model Fit on Train and Test Data

In [ ]:
lr = LinearRegression()
lr.fit(x_train[['engine-size_l', 'horsepower_l', 'highway-mpg_l']], y_train)

In [ ]:
yhat_train = lr.predict(x_train[['engine-size_l', 'horsepower_l', 'highway-mpg_l']])
yhat_train[0:5]

In [ ]:
yhat_test = lr.predict(x_test[['engine-size_l', 'horsepower_l', 'highway-mpg_l']])
yhat_test[0:5]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
def DistributionPlot(RedFunction, BlueFunction, RedName, BlueName, Title):
    width = 6
    height = 4
    plt.figure(figsize=(width, height))

    ax1 = sns.distplot(RedFunction, hist=False, color="r", label=RedName)
    ax2 = sns.distplot(BlueFunction, hist=False, color="b", label=BlueName, ax=ax1)

    plt.title(Title)
    plt.xlabel('Price (in logs)')
    plt.ylabel('Proportion of Cars')

    plt.show()
    plt.close()

In [ ]:
Title = 'Distribution Plot of Predicted vs Actual Price (in Logs) Using Training Data'
DistributionPlot(y_train, yhat_train, "Actual Values (Train)", "Predicted Values (Train)", Title)

In [ ]:
Title='Distribution Plot of Predicted vs Actual Price (in Logs) Using Test Data'
DistributionPlot(y_test, yhat_test, "Actual Values (Test)", "Predicted Values (Test)", Title)